In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# Описание проектной задачи

Распознавание чисел на изображениях — это задача многоклассовой классификации. В качестве объекта, для которого необходимо предсказать класс, выступает изображение. Возможных классов столько, сколько в данных представлено чисел, которые необходимо научиться распознавать.

В качестве данных для решения задачи будут использоваться изображения из базы данных MNIST — классического набора данных для создания и проверки качества работы алгоритмов компьютерного зрения.

MNIST состоит из чёрно-белых изображений рукописных цифр. В базе представлены изображения цифр от 0
до 9. Каждое изображение имеет разрешение 28×28 пикселей.

В данных, которые будут использоваться для решения задачи, представлено 42 000 изображений. Это подмножество всех данных, доступных в MNIST.

In [10]:
# чтение и вывод данных
data = pd.read_csv('images.csv')
data.head(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Изображение размером **28 × 28** представляется в виде последовательности из **784** чисел — по одному числу на каждый пиксель изображения. Каждое число находится в диапазоне от **0** до **255**. Чем больше число, тем выше интенсивность пикселя. Нулевой интенсивности соответствует абсолютно чёрный пиксель, максимальной — абсолютно белый.

В таблице есть колонки **pixel0**, **pixel1** и так далее до **pixel783**, каждая из которых соответствует значению конкретного пикселя изображения. Также в таблице есть колонка **label**, в которой находится информация о том, какая цифра представлена на изображении.